In [1]:
import pandas as pd
import yfinance as yf
import os
import sys
import requests
sys.path.append("..")
from src.utils import*
from bs4 import BeautifulSoup

In [2]:
#demo
symbol = "NVDA"
df = daily_tickerFetcher(symbol=symbol,start="2015-08-19",end="2025-08-19")
df

[*********************100%***********************]  1 of 1 completed

loading data from yfinance is successful


Price,date,adj close,close,high,low,open,volume,ticker
0,2015-08-19,0.558726,0.574750,0.582750,0.568500,0.576250,249248000,NVDA
1,2015-08-20,0.538555,0.554000,0.571000,0.551250,0.568250,426932000,NVDA
2,2015-08-21,0.521786,0.536750,0.559250,0.536250,0.550000,382664000,NVDA
3,2015-08-24,0.503316,0.517750,0.545750,0.500000,0.506750,682284000,NVDA
4,2015-08-25,0.492865,0.507000,0.542500,0.507000,0.540500,622684000,NVDA
...,...,...,...,...,...,...,...,...
2509,2025-08-12,183.160004,183.160004,184.479996,179.460007,182.960007,145485700,NVDA
2510,2025-08-13,181.589996,181.589996,183.970001,179.350006,182.619995,179871700,NVDA
2511,2025-08-14,182.020004,182.020004,183.020004,179.460007,179.750000,129554000,NVDA
2512,2025-08-15,180.449997,180.449997,181.899994,178.039993,181.880005,156602200,NVDA


In [3]:
#scrape S&P500 benchmark in the past 5 years
bench = "^GSPC"
start = "2020-08-19"
end = "2025-08-19"
benchmark = daily_tickerFetcher(symbol=bench,start=start,end=end)
benchmark.to_csv("../data/raw/benchmark_hist_5y.csv",index=False)
benchmark

[*********************100%***********************]  1 of 1 completed

loading data from yfinance is successful


Price,date,adj close,close,high,low,open,volume,ticker
0,2020-08-19,3374.850098,3374.850098,3399.540039,3369.659912,3392.510010,3679480000,^GSPC
1,2020-08-20,3385.510010,3385.510010,3390.800049,3354.689941,3360.479980,3431040000,^GSPC
2,2020-08-21,3397.159912,3397.159912,3399.959961,3379.310059,3386.010010,3505010000,^GSPC
3,2020-08-24,3431.280029,3431.280029,3432.090088,3413.129883,3418.090088,3743410000,^GSPC
4,2020-08-25,3443.620117,3443.620117,3444.209961,3425.840088,3435.949951,3627650000,^GSPC
...,...,...,...,...,...,...,...,...
1250,2025-08-12,6445.759766,6445.759766,6446.549805,6385.759766,6395.169922,5135300000,^GSPC
1251,2025-08-13,6466.580078,6466.580078,6480.279785,6445.020020,6462.669922,5195950000,^GSPC
1252,2025-08-14,6468.540039,6468.540039,6473.919922,6441.069824,6453.459961,4462050000,^GSPC
1253,2025-08-15,6449.799805,6449.799805,6481.339844,6441.850098,6477.379883,4575060000,^GSPC


In [4]:
#scrape s&p500 component stocks from wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
headers = {"User-Agent":"AFE-Project"}
r = requests.get(url=url,headers=headers)
soup = BeautifulSoup(r.text,"html.parser")
table = soup.find_all("table")
main_table = table[0]
rows = [[c.get_text(strip=True) for c in tr.find_all(["th","td"])] for tr in main_table.find_all("tr") if tr.find_all(["th","td"])]
header, *data = [r for r in rows if r]
df_scrape = pd.DataFrame(data, columns=header)
df_scrape.to_csv("../data/raw/S&P500_stock_list.csv",index=False)
df_scrape

,Symbol,Security,GICSSector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,0001524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,1927


In [ ]:
#scrape all the S&P500 component stocks
stock_list = df_scrape["Symbol"].str.replace(".", "-", regex=False)
idx = 0
df_all = []
stock_list = stock_list.to_list()[idx:]
try:
    for symbol in stock_list:
        df = daily_tickerFetcher(symbol=symbol,start=start,end=end)
        df_all.append(df)
        idx+=1
    df_all = pd.concat(df_all,ignore_index=True)
except:
        print("Error occurred at index:", idx)
        print("Error code:",symbol)


In [6]:
df_all

Price,date,adj close,close,high,low,open,volume,ticker
0,2020-08-19,113.032112,137.098663,138.536789,136.939804,137.909698,2407309,MMM
1,2020-08-20,112.342758,136.262543,136.839462,135.769226,136.304352,2380399,MMM
2,2020-08-21,112.488823,135.209030,136.028427,134.397995,134.556854,2398698,MMM
3,2020-08-24,113.754852,136.730774,137.081940,134.740799,135.075256,3114862,MMM
4,2020-08-25,113.455757,136.371231,138.168900,135.802673,137.842804,2318446,MMM
...,...,...,...,...,...,...,...,...
626570,2025-08-12,149.770004,149.770004,149.880005,148.309998,148.910004,2578500,ZTS
626571,2025-08-13,152.440002,152.440002,152.759995,148.919998,149.619995,3331100,ZTS
626572,2025-08-14,153.059998,153.059998,153.360001,151.169998,151.570007,2565100,ZTS
626573,2025-08-15,154.110001,154.110001,155.100006,152.979996,153.720001,3705700,ZTS


In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626575 entries, 0 to 626574
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date       626575 non-null  datetime64[ns]
 1   adj close  626575 non-null  float64       
 2   close      626575 non-null  float64       
 3   high       626575 non-null  float64       
 4   low        626575 non-null  float64       
 5   open       626575 non-null  float64       
 6   volume     626575 non-null  int64         
 7   ticker     626575 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 38.2+ MB


In [8]:
#save stock hist data to path ../data/raw
df_all.to_csv("../data/raw/S&P500_stock_hist_5y.csv",index=False)